In [42]:
from sqlalchemy import create_engine
import geopandas as gp
import multiprocessing
import zipfile as zp
import pandas as pd
import numpy as np
import shutil
import os
from IPython.display import clear_output, display
import time
import concurrent.futures as cf
import psycopg2
from datetime import datetime
import geoalchemy2
#engine.dispose()


df = pd.read_csv(r'E:\combined.csv')
shp_df = df[df.Ext == 'shp']
shp_df.head()



# Function to read shape files into a temp directory
def shp_zip_read(full_path):
    os.makedirs(r"E:\New folder", exist_ok=True)
    zip_file_path = full_path.split('.zip\\')[0] + '.zip'
    shp_file_path = full_path.split('.zip\\')[1]
    print(shp_file_path)
    archive = zp.ZipFile(zip_file_path)
    if '/' not in shp_file_path:
        for file in archive.namelist():
            if '/' not in file:
                archive.extract(file, r"E:\New folder")
        shp_folder_path = ''
    else:
        shp_folder_path = '/'.join(shp_file_path.split('/')[:-1])+'/'
        for file in archive.namelist():
            if file.startswith(shp_folder_path):
                archive.extract(file, r"E:\New folder")
    gdf = gp.read_file("E:\\New folder\\{}".format(shp_folder_path))
    shutil.rmtree(r'E:\New folder', ignore_errors=True)
    return gdf


# Function to dump shape files into postgres db 
def shape_file_dump(file_name, file_path, engine):
    
    #connection = engine.connect()
    
    # Reading the shape file
    if '.zip' in file_path:
        gdf = shp_zip_read(file_path)
    else:
        gdf = gp.read_file(file_path)

    #Import shapefile to databse
    #print(file_name)
    gdf.to_postgis(name=file_name, con=engine, schema="postgis")
    
    #connection.close()
    

# Deriving a unique file name to create table in the database 
import hashlib
class FilePathShortener:
    def __init__(self):
        self.paths = {}
    
    def shorten_file_path(self, file_path):
        if file_path in self.paths:
            return self.paths[file_path]
        
        file_hash = hashlib.md5(file_path.encode()).hexdigest()
        shortened_path = file_hash[:8]
        
        self.paths[file_path] = shortened_path
        return shortened_path
    
    def decode_shortened_path(self, shortened_path):
        for path, short_path in self.paths.items():
            if short_path == shortened_path:
                return path
        
        return None

shortener = FilePathShortener()

 

"""file_path = '\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\Backup_Del\\Transfer-data\\Sent-Analysis\\shp\\0.India_state_shp\\india state_lcc.shp'
shortened_path = shortener.shorten_file_path(file_path)
print(shortened_path)"""

shortened_path = shp_df.File_path.apply(shortener.shorten_file_path) 
new_file_name = shp_df.File_Name+'_'+shortened_path
shp_df['unique_file_name']=new_file_name
        
   


# Getting the current date and time
#dt = datetime.now()
# getting the timestamp
#ts = datetime.timestamp(dt)
#print("Date and time is:", dt)
#current_time = dt.strftime("%d-%m-%Y %H:%M:%S")

    
if __name__ == '__main__':
    # Number of processes to run in parallel
    num_processes = multiprocessing.cpu_count()
        
    #DB details
    host = 'localhost'
    port = '5432'
    database = 'postgis_33'
    user = 'postgres'
    password = 'wadhwanikdss'

    conn = f"postgresql://{user}:{password}@{host}:{port}/{database}"
    engine = create_engine(conn)
            
#     # Create a list to store the process instances
#     processes = []

#     for _ in range(num_processes):
#         p = multiprocessing.Process(target=shape_file_dump, args=(list(shp_df.File_Name), list(shp_df.File_path), engine))
#         p.start()
#         processes.append(p)

#     # Wait for all processes to finish
#     for p in processes:
#         p.join()
    
    j = 0
    
    
      
for i in range(3833,shp_df.shape[0]):
    #initialising empty data frame to store errors
    df = pd.DataFrame(columns=['iteration','unique_file_name', 'file_path', 'error', 'error_type']) 
        
    
    if '.zip' not in shp_df.File_path.iloc[i]:  
        continue
    
    file_present = False
    #print('waiting for shared drive access ')
    while file_present == False:
        j=j+1
        file_check = r'\\192.168.1.151\ncfc_data3\CHAMAN\mango\East Champaran\analysis\merge_mango_echamparan.shp'
        if os.path.exists(file_check) or os.path.exists(file_check.replace(".151",".152")):
            print(f'Executing {i}st file')
            File_path = shp_df.File_path.iloc[i]
            
            if os.path.exists(File_path) == False:
                File_path = File_path.replace(".151",".152") 
            
            # Getting the current date and time
            dt = datetime.now()
           # getting the timestamp
            ts = datetime.timestamp(dt)
            current_time = dt.strftime("%d-%m-%Y %H:%M:%S")

            #Appending  unique_file_name column with current timestamp
            pg_file_name = shp_df.unique_file_name.iloc[i] +'_' +current_time
            shp_df.unique_file_name.iloc[i] = pg_file_name


            try:
                 shape_file_dump(shp_df.unique_file_name.iloc[i], File_path, engine)
        #print(shp_df.unique_file_name.iloc[i])

            except Exception as e:
                        error=e
                        print(error, File_path)
             #if 'Identifier' not in str(error):
            df.loc[len(df.index)] = [i,pg_file_name, File_path, error, type(error)]

            df.to_csv(f"C:/Users/user/Documents/Shape_file_DB_Upload_Error_tracker(Zippedfiles).csv" , mode='a', index=False, header=False)

                #if type(error) == fiona.errors.DriverError:
            file_present = True
            break 
            
        else:
            print('waiting for shared drive access ') 
            time.sleep(40)
    if j == 1000:
        time.sleep(3)
        clear_output(wait=True)
        display('Output cleared')
        print(i)
        j = 0     


print("Data dumped successfully into the PostGIS database.")
    
engine.dispose()

C:\Users\user\AppData\Local\Temp\ipykernel_17068\1832265514.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df['unique_file_name']=new_file_name


Executing 3833st file
Damoh_GramPanchayats.shp
Executing 3834st file
Dewas_GramPanchayats.shp


C:\Users\user\AppData\Local\Temp\ipykernel_17068\1832265514.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


Executing 3835st file
Raisen_GramPanchayats.shp


C:\Users\user\AppData\Local\Temp\ipykernel_17068\1832265514.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


...

Identifier 'idx_BhuvanFASAL20230203112849_a6ed2321_21-06-2023 21:51:27_geometry' exceeds maximum length of 63 characters \\192.168.1.152\ncfc_data3\MNCFC GT Repository\2022\Haryana\HR_01NOV22_31DEC22\BhuvanFASAL20230203112849.zip\shape/BhuvanFASAL20230203112849.shp
Executing 98873st file
shape/BhuvanFASAL2023011963753.shp


C:\Users\user\AppData\Local\Temp\ipykernel_17068\1832265514.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


Identifier 'idx_BhuvanFASAL2023011963753_81d24226_21-06-2023 21:51:31_geometry' exceeds maximum length of 63 characters \\192.168.1.152\ncfc_data3\MNCFC GT Repository\2022\Himachal Pradesh\HP_01JAN22_31DEC22\BhuvanFASAL2023011963753.zip\shape/BhuvanFASAL2023011963753.shp
Executing 98874st file
shape/BhuvanFASAL2023011970208.shp


C:\Users\user\AppData\Local\Temp\ipykernel_17068\1832265514.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.unique_file_name.iloc[i] = pg_file_name


Identifier 'idx_BhuvanFASAL2023011970208_e1b43835_21-06-2023 21:51:31_geometry' exceeds maximum length of 63 characters \\192.168.1.152\ncfc_data3\MNCFC GT Repository\2022\Jharkhand\JH_01JAN22_31DEC22\BhuvanFASAL2023011970208.zip\shape/BhuvanFASAL2023011970208.shp
Data dumped successfully into the PostGIS database.


In [38]:
print(i)

3832


In [12]:
for i in range(shp_df.shape[0]):
    shp_df.File_path.iloc[i] = shp_df.File_path.iloc[i].replace(".151",".152");
    

C:\Users\user\AppData\Local\Temp\ipykernel_17068\3535022495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shp_df.File_path.iloc[i] = shp_df.File_path.iloc[i].replace(".151",".152");


In [41]:
shp_df.File_path.iloc[3827]

'\\\\192.168.1.151\\ncfc_data3\\Neetu Mam (IP 26 Data Backup 17 April 23)\\5-5-2022_office_work\\PMFBY_Updated\\Shape_Files\\Agrotech Rabi 20-21 boundary\\Karnataka\\Raichur_GramPanchayats.zip\\Raichur_GramPanchayats.shp'